In [21]:
import json
import numpy as np
import tensorflow as tf
import nltk
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation, Dropout
# from tensorflow.keras.optimizers import SGD


In [15]:
with open('intents.json') as intention:
    intents = json.load(intention)
# print(intents)

In [16]:
words = []
tag = []
docs = []
ignore=['!','?','.....',',','.']

In [17]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        token = nltk.word_tokenize(pattern)
        words.extend(token)
        docs.append((token,intent['tag']))
        if intent['tag'] not in tag:
            tag.append(intent['tag'])



In [18]:
print(words)
print(tag)
print(docs)

['Hi', 'Halo', '!', 'Ada', 'orang', 'kah', '?', 'Salam', 'kenal', '!', 'Halo', ',', 'aku', '.....', 'Bye', 'Dadah', 'Sampai', 'jumpa', '!', 'See', 'you', 'laterrr', 'Thanks', 'Thank', 'you', 'Terima', 'kasih', '!', 'Ada', 'apa', 'sih', 'jenis', 'kopi', '?', 'Jenis', 'kopi', 'di', 'Indonesia', 'ada', 'apa', 'aja', '?', 'Jenis', 'kopi', 'ada', 'apa', 'aja', '?', 'Bagaimana', 'cara', 'brew', 'kopi', '?', 'Bagaimana', 'teknik', 'brew', 'kopi', '?', 'Bagaimana', 'cara', 'membuat', 'kopi', 'yang', 'baik', '?', 'Apakah', 'kopi', 'punya', 'rasa-rasa', 'lain', '?', 'Beritahu', 'saya', 'tentang', 'rasa', 'kopi', 'Rasa', 'apa', 'saja', 'yang', 'bisa', 'saya', 'temukan', 'di', 'kopi', '?', 'Berapa', 'tinggi', 'kafein', 'di', 'kopi', '?', 'Tingkatan', 'kafein', 'di', 'kopi', 'Apakah', 'kopi', 'memiliki', 'kafein', 'yang', 'tinggi', '?']
['greeting', 'goodbye', 'thanks', 'types', 'brewing', 'flavors', 'caffeine']
[(['Hi'], 'greeting'), (['Halo', '!'], 'greeting'), (['Ada', 'orang', 'kah', '?'], 'gre

In [19]:
lemmatizer = nltk.WordNetLemmatizer()
words = [lemmatizer.lemmatize(token.lower()) for token in words if token not in ignore]
words = sorted(list(set(words)))

In [20]:
print(words)

['ada', 'aja', 'aku', 'apa', 'apakah', 'bagaimana', 'baik', 'berapa', 'beritahu', 'bisa', 'brew', 'bye', 'cara', 'dadah', 'di', 'halo', 'hi', 'indonesia', 'jenis', 'jumpa', 'kafein', 'kah', 'kasih', 'kenal', 'kopi', 'lain', 'laterrr', 'membuat', 'memiliki', 'orang', 'punya', 'rasa', 'rasa-rasa', 'saja', 'salam', 'sampai', 'saya', 'see', 'sih', 'teknik', 'temukan', 'tentang', 'terima', 'thank', 'thanks', 'tinggi', 'tingkatan', 'yang', 'you']


In [50]:
training_list = []
output_nan = [0] * len(tag)

for doc in docs:
    bow = []
    pattern = doc[0]
    pattern = [lemmatizer.lemmatize(sentence.lower()) for sentence in pattern]
    for word in words:
        bow.append(1) if word in pattern else bow.append(0)

    output = list(output_nan)
    output[tag.index(doc[1])] = 1
    bow_array = np.array(bow)
    output_array = np.array(output)

    training_list.append((bow_array, output_array))
   

train_x = np.array([item[0] for item in training_list])
train_y = np.array([item[1] for item in training_list])

train_x = np.stack(train_x)
train_y = np.stack(train_y)


In [66]:
# print(training_list)
print(train_x)
# print(train_y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]]


In [55]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128,input_shape=(len(train_x[0]),),activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]),activation='softmax')
])

optimizers = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=optimizers,metrics=['acc'])

In [56]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

In [57]:
callbacks = myCallback()

In [68]:
chatbot_model_1 = model.fit(train_x,train_y,epochs=200,batch_size=5,verbose=1)

Epoch 1/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1478 - acc: 1.0000
Epoch 2/200
5/5 [==============================] - 0s 981us/step - loss: 0.0712 - acc: 1.0000
Epoch 3/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0648 - acc: 1.0000
Epoch 4/200
5/5 [==============================] - 0s 7ms/step - loss: 0.0228 - acc: 1.0000
Epoch 5/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0410 - acc: 1.0000
Epoch 6/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0467 - acc: 1.0000
Epoch 7/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0247 - acc: 1.0000
Epoch 8/200
5/5 [==============================] - 0s 2ms/step - loss: 0.1653 - acc: 0.9583
Epoch 9/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0728 - acc: 0.9583
Epoch 10/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0660 - acc: 1.0000
Epoch 11/200
5/5 [==============================] - 0s 2ms/step - loss: 0.061

In [ ]:
import pickle 

